# 'ThugStyle' Threat Hunt

In [89]:
# Import required libraries
import paramiko
from datetime import datetime
from getpass import getpass
import os
import re

In [90]:
# Create an SSH connection with the remote server.

# Host information
host = "192.168.6.71"
port = 2222
username = "reed.simon"

print(f"[-] Connecting via SSH to {username}@{host}:{port}...")

# Create the password prompt
thePass = getpass(prompt="[+] Please enter your SSH password: ")
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)
    print("[-] Connection established.")

except paramiko.AuthenticationException:
    print("[!] Authentication failed.")

[-] Connecting via SSH to reed.simon@192.168.6.71:2222...
[-] Connection established.


In [91]:
# Upload kraken to the remote host

sftp = ssh.open_sftp()

# Check if the file already exists
try:
    sftp.stat("/home/reed.simon/kraken")
    print("[-] Skipping kraken upload, binary already exists.")
except IOError:
    # The file does not exist because we cannot stat it
    # Therefore, copy the file to the remote host
    print("[-] Uploading kraken to the remote host...")
    sftp.put("./kraken", "/home/reed.simon/kraken")
    print("[-] Upload successful")

# sftp.close()

[-] Skipping kraken upload, binary already exists.


In [92]:
# Run the kraken script on the remote server

# Make kraken script executable
ssh.exec_command("chmod +x /home/reed.simon/kraken")

# Command to use when running kraken
kraken_command = "sudo -S /home/reed.simon/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin"

# Run and get the output from kraken
print("[-] Running kraken on the remote host...")
stdin, stdout, stderr = ssh.exec_command(kraken_command)

# For running command with sudo, user password must be entered
stdin.write(password + "\n")

# Get results from stdout
lines = stdout.readlines()

print("[-] Execution of kraken completed")

[-] Running kraken on the remote host...
[-] Execution of kraken completed


In [93]:
# Write output to logfile with timestamp
def write_log(service, output):
    print(f"[-] Writing kraken output to logfile {service}_output.txt")
    with open(f"{service}_output.txt", "a") as f:
        f.write(f"\n### {service} output - ")
        f.write(datetime.now().strftime("%Y-%m-%d %H:%M:%S") + " ###\n")
        f.write(output)

In [94]:
# Write kraken logfile

# Remove ANSI escape sequences from output
# Credit for regex:
# https://stackoverflow.com/questions/14693701/how-can-i-remove-the-ansi-escape-sequences-from-a-string-in-python
ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
output = ansi_escape.sub("", "".join(lines))

# Write output to a logfile
write_log("kraken", output)

[-] Writing kraken output to logfile kraken_output.txt


In [95]:
# Get only pids for detected processes to pipe into lsof

# Match on pid (as a capture group), if lookahead does not find process snapd
pids = re.findall(r"pid=(\d+)(?!.*snapd)", output)

print(f"[-] Detected {len(pids)} process(es) with pid(s) {list(map(int, pids))}")

[-] Detected 1 process(es) with pid(s) [3138]


In [96]:
# Run lsof on detected processes
print(f"[-] Getting list of open files for {len(pids)} process(es)...")

lsof_command = f"sudo -S lsof -p {','.join(pids)}"

# Run and get the output from lsof
stdin, stdout, stderr = ssh.exec_command(lsof_command)

# For running command with sudo, user password must be entered
stdin.write(password + "\n")

lsof_output = "".join(stdout.readlines())

# Write output to a logfile
write_log("lsof", lsof_output)

[-] Getting list of open files for 1 process(es)...
[-] Writing kraken output to logfile lsof_output.txt


In [98]:
# Process lsof output to check for x.xxxxx usernames
print("[-] Checking for suspicious usernames in lsof output")

# Create ThugStyle username format regex
thugstyle_username = re.compile(r"^[A-Za-z]\.[A-Za-z]+")

sus_usernames = []

# Create output directory if it doesn't exist
if not os.path.exists("./output"):
    os.mkdir("output")

# Iterate through lsof output to check for usernames
for line in lsof_output.splitlines():
    # Match on suspicious username pattern
    if re.search(thugstyle_username, line.split()[2]):
        fields = line.split()
        # Check for presence of username in the list, to avoid duplicate output
        if fields[2] not in sus_usernames:
            print(f"[!] Suspicious username found in lsof output: {fields[2]}")
            sus_usernames.append(fields[2])
        # Check lsof entry for download requirements
        if fields[3] != "DEL" and fields[4] == "REG":
            out_filename = "output/" + fields[8].split('/')[-1]
            # Check that file hasn't already been downloaded
            if not os.path.exists(out_filename):
                print(f"[!] Downloading suspicious file: {fields[8]}")
                sftp.get(fields[8], out_filename)
            else:
                print(f"[-] Skipping file {fields[8]}, already downloaded")

print("[-] Download(s) complete")

[-] Checking for suspicious usernames in lsof output
[!] Suspicious username found in lsof output: p.dalton
[-] Skipping file /usr/local/bin/ls, already downloaded
[-] Skipping file /home/p.dalton/test-lk/.d.log, already downloaded
[-] Skipping file /home/p.dalton/test-lk/.d.log, already downloaded
[-] Skipping file /home/p.dalton/test-lk/d.pid, already downloaded
[-] Download(s) complete


In [99]:
# Check other locations on the system for ThugStyle activity
search_commands = ["ps aux", "cat /etc/passwd", "sudo -S cat /var/log/auth.log"]

# Redefine thugstyle regex to work with multi-field lines
thugstyle_username = re.compile(r"(^|[^A-Za-z])([A-Za-z]\.[A-Za-z]+)")

for command in search_commands:
    # Print command being run
    print(f"[-] Running command: {command}")
    
    # Run and get the output for each command
    stdin, stdout, stderr = ssh.exec_command(command)
    
    # Enter password if command uses sudo
    if "sudo" in command:
        stdin.write(password + "\n")
    
    command_output = "".join(stdout.readlines())

    # Trim output to only results related to ThugStyle users
    thugstyle = re.findall(thugstyle_username, output)

    for thug in thugstyle:
        if thug not in sus_usernames:
            print(f"[!] Suspicious username found: {thug}")
            sus_usernames.append(thug)

    # Output full suspicious line
    for line in command_output.splitlines():
        if re.search(thugstyle_username, line):
            print(line)

[-] Running command: ps aux
p.dalton    3025  0.0  0.1  18408 18352 ?        Ss   Feb26   0:00 /lib/systemd/systemd --user
p.dalton    3027  0.0  0.6 104268 104256 ?       S    Feb26   0:00 (sd-pam)
p.dalton    3138  0.0  0.0 1003532 4892 ?        Ssl  Feb26   0:01 [go-daemon sample]
[-] Running command: cat /etc/passwd
p.dalton:x:1001:1001::/home/p.dalton:/bin/bash
[-] Running command: sudo -S cat /var/log/auth.log
Mar  7 00:42:48 sys320 sudo: reed.simon : TTY=unknown ; PWD=/home/reed.simon ; USER=root ; COMMAND=/usr/local/bin/ls -la /home/p.dalton
Mar  7 00:42:49 sys320 sudo: reed.simon : TTY=unknown ; PWD=/home/reed.simon ; USER=root ; COMMAND=/usr/local/bin/ls -la /home/g.yaml
Mar  7 00:44:44 sys320 sudo: reed.simon : TTY=unknown ; PWD=/home/reed.simon ; USER=root ; COMMAND=/usr/local/bin/ls -la /home/p.dalton
Mar  7 00:44:44 sys320 sudo: reed.simon : TTY=unknown ; PWD=/home/reed.simon ; USER=root ; COMMAND=/usr/local/bin/ls -la /home/g.yaml


In [100]:
# List contents of suspicious user's home directories
for username in sus_usernames:
    print(f"[-] Listing contents of home directory for {username}")
    stdin, stdout, stderr = ssh.exec_command(f"/usr/bin/ls -la /home/{username}")
    print("".join(stdout.readlines()))

[-] Listing contents of home directory for p.dalton
total 524
drwxr-xr-x  4 p.dalton p.dalton   4096 Mar  3 15:18 .
drwxr-xr-x 43 root     root       4096 Mar  6 17:06 ..
-rwxr-xr-x  1 p.dalton p.dalton     96 Feb 27 01:17 .bash_history
drwxr-xr-x  2 p.dalton p.dalton   4096 Feb 26 23:50 .cache
-rwxr-xr-x  1 p.dalton p.dalton    170 Mar  3 15:18 .d.log
-rwxr-xr-x  1 p.dalton p.dalton 511476 Feb 18 15:42 aqwerpuqwerj
drwxr-xr-x  2 p.dalton p.dalton   4096 Feb 26 23:54 test-lk



In [101]:
# Close the SSH session
ssh.close()